# Sign Language Recognition using Deep Learning

โปรเจกต์นี้สร้าง AI Model สำหรับจำแนก Sign Language โดยใช้ Sign Language MNIST Dataset

## เนื้อหา
1. ติดตั้ง Libraries และโหลด Dataset
2. สำรวจข้อมูล (Exploratory Data Analysis)
3. เตรียมข้อมูล (Data Preprocessing)
4. สร้าง Model แบบ CNN
5. สร้าง Model แบบ Transfer Learning
6. Train และประเมินผล Models
7. บันทึก Model และทดสอบ

## 1. Import Libraries และ Download Dataset

**Alternative (สำหรับ Google Colab)**:  
หากรันบน Colab และต้องการติดตั้งด้วยคำสั่งง่ายๆ ให้ใช้:
```python
!pip install -q kagglehub tensorflow numpy pandas matplotlib seaborn scikit-learn opencv-python Pillow
```

In [ ]:
# ติดตั้ง dependencies ตาม requirements.txt
# สำหรับ Google Colab หรือสภาพแวดล้อมใหม่

import sys
import subprocess

def install_requirements():
    """ติดตั้ง packages จากไฟล์ requirements.txt"""
    try:
        print("กำลังติดตั้ง dependencies...")
        print("="*60)
        
        # ติดตั้งจากไฟล์ requirements.txt
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt", "-q"])
        
        print("="*60)
        print("✅ ติดตั้ง dependencies สำเร็จ!")
        print("\nหมายเหตุ: ถ้าติดตั้งไว้แล้ว จะข้ามการติดตั้งซ้ำโดยอัตโนมัติ")
        
    except FileNotFoundError:
        print("⚠️  ไม่พบไฟล์ requirements.txt")
        print("กำลังติดตั้ง packages หลักๆ...")
        
        # ติดตั้ง packages หลักๆ ถ้าไม่มีไฟล์ requirements.txt
        packages = [
            "kagglehub",
            "tensorflow>=2.13.0",
            "numpy>=1.24.0",
            "pandas>=2.0.0",
            "matplotlib>=3.7.0",
            "seaborn>=0.12.0",
            "scikit-learn>=1.3.0",
            "opencv-python>=4.8.0",
            "Pillow>=10.0.0"
        ]
        
        for package in packages:
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
                print(f"✅ ติดตั้ง {package}")
            except:
                print(f"❌ ติดตั้ง {package} ล้มเหลว")
        
        print("\n✅ ติดตั้ง dependencies เสร็จสิ้น!")
    
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาด: {str(e)}")
        print("\nคุณสามารถติดตั้งด้วยตนเองโดยรันคำสั่ง:")
        print("!pip install -r requirements.txt")

# รันการติดตั้ง
install_requirements()

## 0. ติดตั้ง Dependencies (ถ้ายังไม่ได้ติดตั้ง)

**หมายเหตุ**: ถ้าคุณรัน Notebook ใน Google Colab หรือสภาพแวดล้อมใหม่ ให้รัน cell นี้ก่อน  
ถ้าติดตั้ง dependencies ไว้แล้ว สามารถข้ามไปยัง Section 1 ได้เลย

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2

# ML Utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Kaggle
import kagglehub

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Download dataset from Kaggle
print("Downloading Sign Language MNIST dataset...")
path = kagglehub.dataset_download("datamunge/sign-language-mnist")
print(f"Path to dataset files: {path}")

In [ ]:
# โหลดข้อมูล
train_df = pd.read_csv(os.path.join(path, 'sign_mnist_train.csv'))
test_df = pd.read_csv(os.path.join(path, 'sign_mnist_test.csv'))

print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")
print(f"\nFirst few rows:")
print(train_df.head())

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# แสดงข้อมูลสถิติ
print("Dataset Information:")
print(f"Number of training samples: {len(train_df)}")
print(f"Number of test samples: {len(test_df)}")
print(f"Number of features: {train_df.shape[1] - 1}")  # ลบ label column
print(f"Number of classes: {train_df['label'].nunique()}")
print(f"\nClass distribution in training set:")
print(train_df['label'].value_counts().sort_index())

In [ ]:
# สร้าง label mapping (A-Z ยกเว้น J และ Z)
# Dataset มี 24 คลาส (0-24) ที่แทน A-Y ยกเว้น J
label_map = {i: chr(65 + i) if i < 9 else chr(65 + i + 1) for i in range(24)}
# 0=A, 1=B, ..., 8=I, 9=K, ..., 24=Y
print("Label Mapping:")
print(label_map)

In [ ]:
# แสดง class distribution
plt.figure(figsize=(15, 5))
sns.countplot(data=train_df, x='label')
plt.title('Class Distribution in Training Set')
plt.xlabel('Sign Language Letter')
plt.ylabel('Count')
plt.xticks(range(24), [label_map[i] for i in range(24)])
plt.grid(axis='y', alpha=0.3)
plt.show()

In [ ]:
# แสดงตัวอย่างภาพจากแต่ละคลาส
def display_sample_images(df, n_samples=24):
    fig, axes = plt.subplots(4, 6, figsize=(15, 10))
    axes = axes.flatten()
    
    for i in range(n_samples):
        # เลือกภาพแรกของแต่ละคลาส
        sample = df[df['label'] == i].iloc[0]
        pixels = sample[1:].values.reshape(28, 28)
        
        axes[i].imshow(pixels, cmap='gray')
        axes[i].set_title(f'Label: {label_map[i]} ({i})')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle('Sample Images from Each Class', y=1.02, fontsize=16)
    plt.show()

display_sample_images(train_df)

In [ ]:
# แสดงตัวอย่างหลายภาพจากคลาสเดียวกัน
def display_multiple_samples(df, label_num, n_samples=9):
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    axes = axes.flatten()
    
    samples = df[df['label'] == label_num].sample(n_samples)
    
    for i, (idx, sample) in enumerate(samples.iterrows()):
        pixels = sample[1:].values.reshape(28, 28)
        axes[i].imshow(pixels, cmap='gray')
        axes[i].set_title(f'{label_map[label_num]} - Sample {i+1}')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f'Multiple Samples of Letter "{label_map[label_num]}"', y=1.02, fontsize=16)
    plt.show()

# แสดงตัวอย่างหลายภาพของตัวอักษร 'A' (label 0)
display_multiple_samples(train_df, label_num=0)

## 3. Data Preprocessing

In [ ]:
# แยก features และ labels
X_train = train_df.drop('label', axis=1).values
y_train = train_df['label'].values

X_test = test_df.drop('label', axis=1).values
y_test = test_df['label'].values

# Reshape เป็น 28x28x1 (grayscale)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Normalize pixel values (0-255 -> 0-1)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
# แบ่ง validation set จาก training set (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, 
    test_size=0.2, 
    random_state=42,
    stratify=y_train  # รักษาสัดส่วนของแต่ละคลาส
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# One-hot encode labels
num_classes = 24
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_val_cat = keras.utils.to_categorical(y_val, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)

print(f"One-hot encoded labels shape: {y_train_cat.shape}")

In [ ]:
# Data Augmentation สำหรับ training set
train_datagen = ImageDataGenerator(
    rotation_range=10,          # หมุนภาพ ±10 องศา
    width_shift_range=0.1,      # เลื่อนภาพแนวนอน
    height_shift_range=0.1,     # เลื่อนภาพแนวตั้ง
    zoom_range=0.1,             # ซูมเข้า-ออก
    shear_range=0.1,            # บิดภาพ
    fill_mode='nearest'         # เติมพิกเซลที่ว่าง
)

# Validation/Test ไม่ต้อง augment
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

print("Data augmentation configured successfully!")

In [ ]:
# แสดงตัวอย่าง augmented images
def show_augmented_images(image, n_samples=9):
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    axes = axes.flatten()
    
    # Original image
    axes[0].imshow(image.reshape(28, 28), cmap='gray')
    axes[0].set_title('Original')
    axes[0].axis('off')
    
    # Augmented images
    image_reshaped = image.reshape(1, 28, 28, 1)
    aug_iter = train_datagen.flow(image_reshaped, batch_size=1)
    
    for i in range(1, n_samples):
        aug_image = next(aug_iter)[0]
        axes[i].imshow(aug_image.reshape(28, 28), cmap='gray')
        axes[i].set_title(f'Augmented {i}')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle('Data Augmentation Examples', y=1.02, fontsize=16)
    plt.show()

# แสดงตัวอย่าง augmentation
sample_image = X_train[0]
show_augmented_images(sample_image)

## 4. สร้าง CNN Model (แบบพื้นฐาน)

In [ ]:
def create_cnn_model(input_shape=(28, 28, 1), num_classes=24):
    """
    สร้าง CNN Model พื้นฐานสำหรับจำแนก Sign Language
    """
    model = models.Sequential([
        # Conv Block 1
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Conv Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Conv Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Dense Layers
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        # Output Layer
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# สร้าง model
cnn_model = create_cnn_model()
cnn_model.summary()

In [ ]:
# Compile model
cnn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("CNN Model compiled successfully!")

In [ ]:
# Define Callbacks
callbacks = [
    ModelCheckpoint(
        'best_cnn_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("Callbacks configured!")

In [ ]:
# Train CNN Model
batch_size = 128
epochs = 50

print("Starting training CNN model...")
history_cnn = cnn_model.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=batch_size),
    validation_data=(X_val, y_val_cat),
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
)

print("\nTraining completed!")

In [ ]:
# Plot training history
def plot_training_history(history, title_prefix=''):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy
    ax1.plot(history.history['accuracy'], label='Train Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Val Accuracy')
    ax1.set_title(f'{title_prefix} Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Loss
    ax2.plot(history.history['loss'], label='Train Loss')
    ax2.plot(history.history['val_loss'], label='Val Loss')
    ax2.set_title(f'{title_prefix} Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history_cnn, 'CNN')

## 5. สร้าง Transfer Learning Model (MobileNetV2)

In [ ]:
# เตรียมข้อมูลสำหรับ Transfer Learning (ต้องการ 3 channels)
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_val_rgb = np.repeat(X_val, 3, axis=-1)
X_test_rgb = np.repeat(X_test, 3, axis=-1)

# Resize เป็น 96x96 (ขนาดที่ MobileNetV2 รองรับ)
X_train_resized = tf.image.resize(X_train_rgb, [96, 96]).numpy()
X_val_resized = tf.image.resize(X_val_rgb, [96, 96]).numpy()
X_test_resized = tf.image.resize(X_test_rgb, [96, 96]).numpy()

print(f"Resized training data shape: {X_train_resized.shape}")

In [ ]:
def create_transfer_learning_model(input_shape=(96, 96, 3), num_classes=24):
    """
    สร้าง Transfer Learning Model โดยใช้ MobileNetV2
    """
    # โหลด pre-trained MobileNetV2
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze base model layers
    base_model.trainable = False
    
    # สร้าง model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model

# สร้าง model
transfer_model, base_model = create_transfer_learning_model()
transfer_model.summary()

In [ ]:
# Compile Transfer Learning model
transfer_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Transfer Learning Model compiled successfully!")

In [ ]:
# Callbacks สำหรับ Transfer Learning
tl_callbacks = [
    ModelCheckpoint(
        'best_transfer_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

In [ ]:
# Data Augmentation สำหรับ Transfer Learning
tl_train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    shear_range=0.15,
    horizontal_flip=False,  # ไม่ flip เพราะ Sign Language มีทิศทาง
    fill_mode='nearest'
)

print("Data augmentation for Transfer Learning configured!")

In [ ]:
# Train Transfer Learning Model (Phase 1: Frozen base)
print("Phase 1: Training with frozen base model...")
history_tl = transfer_model.fit(
    tl_train_datagen.flow(X_train_resized, y_train_cat, batch_size=64),
    validation_data=(X_val_resized, y_val_cat),
    epochs=20,
    callbacks=tl_callbacks,
    verbose=1
)

print("\nPhase 1 training completed!")

In [ ]:
# Fine-tuning: Unfreeze ชั้นบนสุดของ base model
print("Phase 2: Fine-tuning...")
base_model.trainable = True

# Freeze ชั้นแรกๆ ไว้ (fine-tune เฉพาะชั้นบนสุด)
for layer in base_model.layers[:100]:
    layer.trainable = False

# Compile ใหม่ด้วย learning rate ต่ำ
transfer_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Trainable layers: {sum([1 for layer in transfer_model.layers if layer.trainable])}")

In [ ]:
# Train Phase 2: Fine-tuning
history_tl_ft = transfer_model.fit(
    tl_train_datagen.flow(X_train_resized, y_train_cat, batch_size=64),
    validation_data=(X_val_resized, y_val_cat),
    epochs=20,
    callbacks=tl_callbacks,
    verbose=1
)

print("\nFine-tuning completed!")

In [ ]:
# Plot Transfer Learning history
plot_training_history(history_tl, 'Transfer Learning (Phase 1)')
plot_training_history(history_tl_ft, 'Transfer Learning (Phase 2 - Fine-tuned)')

## 6. ประเมินผล Models

In [ ]:
# Evaluate CNN Model
print("Evaluating CNN Model...")
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test_cat, verbose=0)
print(f"CNN Test Accuracy: {cnn_accuracy*100:.2f}%")
print(f"CNN Test Loss: {cnn_loss:.4f}")

# Predictions
cnn_predictions = cnn_model.predict(X_test)
cnn_pred_classes = np.argmax(cnn_predictions, axis=1)

In [ ]:
# Evaluate Transfer Learning Model
print("Evaluating Transfer Learning Model...")
tl_loss, tl_accuracy = transfer_model.evaluate(X_test_resized, y_test_cat, verbose=0)
print(f"Transfer Learning Test Accuracy: {tl_accuracy*100:.2f}%")
print(f"Transfer Learning Test Loss: {tl_loss:.4f}")

# Predictions
tl_predictions = transfer_model.predict(X_test_resized)
tl_pred_classes = np.argmax(tl_predictions, axis=1)

In [ ]:
# เปรียบเทียบผลลัพธ์
print("\n" + "="*50)
print("MODEL COMPARISON")
print("="*50)
print(f"{'Model':<30} {'Accuracy':<15} {'Loss':<10}")
print("-"*50)
print(f"{'CNN (Custom)':<30} {cnn_accuracy*100:>6.2f}%        {cnn_loss:>6.4f}")
print(f"{'Transfer Learning (MobileNetV2)':<30} {tl_accuracy*100:>6.2f}%        {tl_loss:>6.4f}")
print("="*50)

In [ ]:
# Classification Report for CNN
print("\nCNN Model - Classification Report:")
print(classification_report(
    y_test, 
    cnn_pred_classes, 
    target_names=[label_map[i] for i in range(24)]
))

In [ ]:
# Classification Report for Transfer Learning
print("\nTransfer Learning Model - Classification Report:")
print(classification_report(
    y_test, 
    tl_pred_classes, 
    target_names=[label_map[i] for i in range(24)]
))

In [ ]:
# Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, title='Confusion Matrix'):
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(16, 14))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=[label_map[i] for i in range(24)],
                yticklabels=[label_map[i] for i in range(24)],
                cbar_kws={'label': 'Count'})
    plt.title(title, fontsize=16)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.show()

# Plot confusion matrices
plot_confusion_matrix(y_test, cnn_pred_classes, 'CNN Model - Confusion Matrix')
plot_confusion_matrix(y_test, tl_pred_classes, 'Transfer Learning Model - Confusion Matrix')

## 7. บันทึก Model และทดสอบ

In [ ]:
# บันทึก models
cnn_model.save('sign_language_cnn_final.keras')
transfer_model.save('sign_language_transfer_final.keras')

print("Models saved successfully!")
print("- sign_language_cnn_final.keras")
print("- sign_language_transfer_final.keras")

In [ ]:
# ฟังก์ชันทดสอบการทำนาย
def predict_sign_language(model, image, model_type='cnn'):
    """
    ทำนาย Sign Language จากภาพ
    
    Args:
        model: trained model
        image: numpy array ของภาพ (28x28x1)
        model_type: 'cnn' หรือ 'transfer'
    """
    # เตรียมภาพ
    if model_type == 'transfer':
        # Convert to RGB and resize
        image_rgb = np.repeat(image, 3, axis=-1)
        image_processed = tf.image.resize(image_rgb, [96, 96]).numpy()
    else:
        image_processed = image
    
    # Reshape สำหรับการทำนาย
    image_input = image_processed.reshape(1, *image_processed.shape)
    
    # ทำนาย
    prediction = model.predict(image_input, verbose=0)
    predicted_class = np.argmax(prediction)
    confidence = prediction[0][predicted_class]
    
    return predicted_class, confidence, prediction[0]

In [ ]:
# ทดสอบด้วยภาพสุ่มจาก test set
def test_random_predictions(n_samples=10):
    """
    ทดสอบการทำนายด้วยภาพสุ่ม
    """
    # สุ่มภาพ
    random_indices = np.random.choice(len(X_test), n_samples, replace=False)
    
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    axes = axes.flatten()
    
    for idx, test_idx in enumerate(random_indices):
        # ภาพจริง
        image = X_test[test_idx]
        true_label = y_test[test_idx]
        
        # ทำนายด้วย CNN
        pred_class_cnn, conf_cnn, _ = predict_sign_language(cnn_model, image, 'cnn')
        
        # ทำนายด้วย Transfer Learning
        pred_class_tl, conf_tl, _ = predict_sign_language(transfer_model, image, 'transfer')
        
        # แสดงผล
        axes[idx].imshow(image.reshape(28, 28), cmap='gray')
        
        title = f"True: {label_map[true_label]}\n"
        title += f"CNN: {label_map[pred_class_cnn]} ({conf_cnn*100:.1f}%)\n"
        title += f"TL: {label_map[pred_class_tl]} ({conf_tl*100:.1f}%)"
        
        # เปลี่ยนสีถ้าทำนายผิด
        color = 'green' if (pred_class_cnn == true_label and pred_class_tl == true_label) else 'red'
        axes[idx].set_title(title, fontsize=10, color=color)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle('Random Test Predictions Comparison', y=1.02, fontsize=16)
    plt.show()

# ทดสอบ
test_random_predictions(10)

In [ ]:
# แสดง Top-3 predictions สำหรับแต่ละภาพ
def show_top_predictions(image_idx, model, model_type='cnn', model_name='Model'):
    """
    แสดง Top-3 predictions พร้อมความมั่นใจ
    """
    image = X_test[image_idx]
    true_label = y_test[image_idx]
    
    pred_class, confidence, all_probs = predict_sign_language(model, image, model_type)
    
    # Top-3
    top_3_idx = np.argsort(all_probs)[-3:][::-1]
    top_3_probs = all_probs[top_3_idx]
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # แสดงภาพ
    ax1.imshow(image.reshape(28, 28), cmap='gray')
    ax1.set_title(f'True Label: {label_map[true_label]}', fontsize=14)
    ax1.axis('off')
    
    # แสดง Top-3 predictions
    labels = [label_map[i] for i in top_3_idx]
    colors = ['green' if top_3_idx[0] == true_label else 'red', 'orange', 'blue']
    
    ax2.barh(labels, top_3_probs, color=colors)
    ax2.set_xlabel('Confidence', fontsize=12)
    ax2.set_title(f'{model_name} - Top 3 Predictions', fontsize=14)
    ax2.set_xlim([0, 1])
    
    for i, (label, prob) in enumerate(zip(labels, top_3_probs)):
        ax2.text(prob + 0.01, i, f'{prob*100:.1f}%', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# ทดสอบกับภาพสุ่ม
random_idx = np.random.randint(0, len(X_test))
show_top_predictions(random_idx, cnn_model, 'cnn', 'CNN Model')
show_top_predictions(random_idx, transfer_model, 'transfer', 'Transfer Learning Model')

## สรุปผลการทดลอง

### Models ที่สร้าง:
1. **CNN Model**: Custom CNN architecture
2. **Transfer Learning Model**: MobileNetV2 + Fine-tuning

### ไฟล์ที่บันทึก:
- `best_cnn_model.keras` - Best CNN model during training
- `best_transfer_model.keras` - Best Transfer Learning model during training
- `sign_language_cnn_final.keras` - Final CNN model
- `sign_language_transfer_final.keras` - Final Transfer Learning model

### วิธีใช้งาน Model:
```python
# โหลด model
model = keras.models.load_model('sign_language_cnn_final.keras')

# เตรียมภาพ (28x28x1, normalized)
image = your_image / 255.0
image = image.reshape(1, 28, 28, 1)

# ทำนาย
prediction = model.predict(image)
predicted_class = np.argmax(prediction)
print(f"Predicted: {label_map[predicted_class]}")
```

### การพัฒนาต่อ:
1. ลองใช้ model architectures อื่นๆ (EfficientNet, ResNet)
2. เพิ่ม data augmentation techniques
3. รวม ensemble models
4. สร้าง web application หรือ mobile app
5. Real-time sign language detection ด้วย webcam

In [ ]:
# สรุปข้อมูล Model
print("\n" + "="*60)
print("PROJECT SUMMARY: SIGN LANGUAGE RECOGNITION")
print("="*60)
print(f"\nDataset: Sign Language MNIST")
print(f"Total Classes: {num_classes} (A-Y, excluding J)")
print(f"\nTraining Samples: {len(X_train)}")
print(f"Validation Samples: {len(X_val)}")
print(f"Test Samples: {len(X_test)}")
print(f"\n{'-'*60}")
print(f"CNN Model Test Accuracy: {cnn_accuracy*100:.2f}%")
print(f"Transfer Learning Test Accuracy: {tl_accuracy*100:.2f}%")
print(f"{'-'*60}")
print(f"\nBest Model: {'Transfer Learning' if tl_accuracy > cnn_accuracy else 'CNN'}")
print(f"Best Accuracy: {max(cnn_accuracy, tl_accuracy)*100:.2f}%")
print("\n" + "="*60)